In [1]:
!pip install -U sqlalchemy
!pip install -U sqlite-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 15.2 MB/s eta 0:00:00m eta 0:00:010:0101
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.1
    Uninstalling SQLAlchemy-2.0.1:
      Successfully uninstalled SQLAlchemy-2.0.1


In [2]:
# simple coalece function
def coalesce4(*args):
    for x in args:
        if x is not None:
            return x

In [2]:
from sqlalchemy import Column
from sqlalchemy import ForeignKey
from sqlalchemy import Integer
from sqlalchemy import String
from sqlalchemy import JSON
from sqlalchemy import Date
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlalchemy import Text # the datatype
from sqlalchemy import text # the function
from sqlalchemy import LargeBinary

import sqlite_utils

from pymarc import MARCReader
import zipfile
import json
from datetime import datetime

engine = create_engine('sqlite:///record_data.db')

Base = declarative_base()

class Record(Base):
    __tablename__ = "record"

    id = Column(Integer, primary_key=True)
    
    title = Column(String, nullable=True)
    
    # 773 - Host Item Entry (R)
    # $a - Main entry heading (NR)
    # $g - Related parts (R)
    # (record['773']['a'], record['773']['g'])
    pub_date = Column(Date, nullable=True)
    pub_name = Column(String, nullable=True)

    # record['260']
    pub_year = Column(String, nullable=True)
    
    citation = Column(String, nullable=True)
    
    # 998 subfield a - "Location"
    location = Column(String, nullable=True)
    
    # # if we want to have a fully seperate table... 
    # # ... don't forget to uncomment the class for it below
    # subjects = relationship(
    #     "Subject", back_populates="record", cascade="all, delete-orphan"
    # )
    
    subjects = Column(JSON, nullable=True)
    
    # 998 subfield b - "Cat. Date"
    cat_date = Column(Date, nullable=True)
    
    # 998 subfield c - "Bib Level"
    bib_level = Column(String, nullable=True)
    
    # 998 subfield d - "Format (MatType)"
    mat_type = Column(String, nullable=True)
    
    # 907 subfield c - "Created"
    create_date = Column(Date, nullable=True)    
    
    # 907 subfield b - "Last Updated"
    last_update = Column(Date, nullable=True)
    
    marc_data = relationship(
        "MarcData", back_populates="record", cascade="all, delete-orphan"
    )
    
    
# class Subject(Base):
#     __tablename__ = "subject"

#     id = Column(Integer, primary_key=True)
#     value = Column(String, nullable=False)
#     occ_num = Column(Integer)
#     record_id = Column(Integer, ForeignKey("record.id"), nullable=False)

#     record = relationship("Record", back_populates="subjects")

class MarcData(Base):
    __tablename__ = "marc_data"
    # def __repr__(self):
    #     return f"Address(id={self.id!r}, email_address={self.email_address!r})"
    id = Column(Integer, primary_key=True)
    record_id = Column(Integer, ForeignKey("record.id"), nullable=False)
    
    marc_in_json = Column(JSON, nullable=True)
    marc_in_text = Column(Text, nullable=True)
    marc_in_binary = Column(LargeBinary, nullable=True)
    
    record = relationship("Record", back_populates="marc_data")

# # create the schema    
Base.metadata.create_all(engine)

In [4]:
with Session(engine) as session:
    with zipfile.ZipFile('2023-01-24_newsdex-full-marc-export.zip') as marc_data_file:
        with marc_data_file.open(marc_data_file.filelist[0]) as fh:
            record_list = []
            # was getting this error 
            # https://github.com/edsu/pymarc/issues/129
            reader = MARCReader(fh, to_unicode=True, force_utf8=True)
            for i, record in enumerate(reader):
                try:
                    subjects=[]
                    for j, s in enumerate([subject.value() for subject in record.subjects()]):
                        # subjects.append(Subject(value=s, occ_num=j))
                        subjects.append(s)
                except:
                    subjects=[]
                
                try:
                    title=record.title()
                except:
                    title=None
                
                # 998 subfield a - "Location"
                try:
                    location=record['998']['a'].format()
                except:
                    location=None

                # 998 subfield b - "Cat. Date"
                try:
                    cat_date=datetime.strptime(record['998']['b'].format(), '%m-%d-%y')
                except:
                    cat_date=None
                    
                # 907 subfield c - "Created"
                try:
                    create_date=datetime.strptime(record['907']['c'].format(), '%m-%d-%y')
                except:
                    create_date=None

                # 907 subfield b - "Last Updated"
                try:
                    last_update=datetime.strptime(record['907']['b'].format(), '%m-%d-%y')
                except:
                    last_update=None

                # 998 subfield c - "Bib Level"
                try:
                    bib_level=record['998']['c'].format()
                except:
                    bib_level=None

                # 998 subfield d - "Format (MatType)"
                try:
                    mat_type=record['998']['d'].format()
                except:
                    mat_type=None
                    
                
                # 773 - Host Item Entry (R)
                # $a - Main entry heading (NR)
                # $g - Related parts (R)
                # (record['773']['a'], record['773']['g'])
                
                # this is the full citation
                try:
                    citation=record['773'].value().format()
                except:
                    citation=None
                
                try:
                    pub_name=record['773']['a'].format()
                except:
                    pub_name=None
                
                try:
                    # datetime.strptime(record['907']['b'], '%m-%d-%y')
                    pub_date=datetime.strptime(record['773']['g'].format()[:10], '%m/%d/%Y')
                except:
                    pub_date=None

                
                # 260$c - Date of publication, distribution, etc. (R)
                try:
                    pub_year=record['260']['c'].format()
                except:
                    pub_year=None
                    
                
                try:
                    marc_data=MarcData(
                        # marc_in_json = Column(JSON, nullable=True)
                        marc_in_json=json.loads(record.as_json()),
                        # print(record_json)
                        
                        # marc_in_binary = Column(LargeBinary, nullable=True)
                        marc_in_binary=record.as_marc21(),
                        
                        # marc_in_text = Column(Text, nullable=True)
                        marc_in_text=str(record).replace('\\', ' ')
                        
                        
                    )
                except:
                    marc_data=MarcData(
                        marc_in_json=None,
                        marc_in_binary=None,
                        marc_in_text=None
                    )
                
                r = Record(
                    title=title,
                    pub_name=pub_name,
                    pub_date=pub_date,
                    pub_year=pub_year,
                    citation=citation,
                    location=location,
                    cat_date=cat_date,
                    create_date=create_date,
                    last_update=last_update,
                    bib_level=bib_level,
                    mat_type=mat_type,
                    
                    # record_json=record_json,
                    subjects=subjects,
                    
                    marc_data=[marc_data],
                )
                record_list.append(r)
                
                if (i % 50000 == 0 and i!=0):
                    session.add_all(record_list)
                    session.commit()
                    del(record_list)
                    record_list = []
                    print(i, '.', end='')
                    # break
                    
            session.add_all(record_list)
            session.commit()
            print(i, '.', end='')

50000 .100000 .150000 .200000 .250000 .300000 .350000 .400000 .450000 .500000 .550000 .600000 .650000 .700000 .750000 .800000 .850000 .900000 .950000 .1000000 .1050000 .1100000 .1150000 .1200000 .1250000 .1300000 .1350000 .1400000 .1450000 .1500000 .1550000 .1600000 .1650000 .1700000 .1750000 .1800000 .1850000 .1886383 .

In [5]:
marc_as_text = "\n".join(
    [f"{field.tag}    {field.value()}" for field in record.fields]
)

print(marc_as_text)

008                                            
245    Public library staff: Instructive talks heard and a social hour enjoyed by employees
260    1900
650    Public Library
650    Public Library, Personnel
650    Hodges, Nathaniel Dana Carlisle
773    Commercial Tribune 10/02/1900 6:6
907    .b28911532 03-06-20 03-06-20
998    2ma 03-06-20 m - - eng 0


In [6]:
record.leader

'00467nam  2200133 a 4500'

In [7]:
# marc_as_text
record_str = str(record).replace('\\', ' ')
print(record_str)

=LDR  00467nam  2200133 a 4500
=008                                          
=245   0$aPublic library staff: Instructive talks heard and a social hour enjoyed by employees
=260    $c1900
=650    $aPublic Library
=650    $aPublic Library, Personnel
=650    $aHodges, Nathaniel Dana Carlisle
=773    $aCommercial Tribune$g10/02/1900 6:6
=907    $a.b28911532$b03-06-20$c03-06-20
=998    $a2ma$b03-06-20$cm$d-$e-$feng$h0



In [8]:
# marc_as_json
record_json = json.loads(record.as_json())
print(record_json)

{'leader': '00467nam  2200133 a 4500', 'fields': [{'008': '                                        '}, {'245': {'subfields': [{'a': 'Public library staff: Instructive talks heard and a social hour enjoyed by employees'}], 'ind1': ' ', 'ind2': '0'}}, {'260': {'subfields': [{'c': '1900'}], 'ind1': ' ', 'ind2': ' '}}, {'650': {'subfields': [{'a': 'Public Library'}], 'ind1': ' ', 'ind2': ' '}}, {'650': {'subfields': [{'a': 'Public Library, Personnel'}], 'ind1': ' ', 'ind2': ' '}}, {'650': {'subfields': [{'a': 'Hodges, Nathaniel Dana Carlisle'}], 'ind1': ' ', 'ind2': ' '}}, {'773': {'subfields': [{'a': 'Commercial Tribune'}, {'g': '10/02/1900 6:6'}], 'ind1': ' ', 'ind2': ' '}}, {'907': {'subfields': [{'a': '.b28911532'}, {'b': '03-06-20'}, {'c': '03-06-20'}], 'ind1': ' ', 'ind2': ' '}}, {'998': {'subfields': [{'a': '2ma'}, {'b': '03-06-20'}, {'c': 'm'}, {'d': '-'}, {'e': '-'}, {'f': 'eng'}, {'h': '0'}], 'ind1': ' ', 'ind2': ' '}}]}


In [9]:
# marc_as_binary
record.as_marc21()

b'00467nam  2200133 a 4500008004100000245008900041260000900130650001900139650003000158650003600188773003900224907003500263998003500298\x1e                                        \x1e 0\x1faPublic library staff: Instructive talks heard and a social hour enjoyed by employees\x1e  \x1fc1900\x1e  \x1faPublic Library\x1e  \x1faPublic Library, Personnel\x1e  \x1faHodges, Nathaniel Dana Carlisle\x1e  \x1faCommercial Tribune\x1fg10/02/1900 6:6\x1e  \x1fa.b28911532\x1fb03-06-20\x1fc03-06-20\x1e  \x1fa2ma\x1fb03-06-20\x1fcm\x1fd-\x1fe-\x1ffeng\x1fh0\x1e\x1d'

In [10]:
# set up the FTS search with datasette now
# # # # set up the full-text-search on record data

# sql = """\
# DROP TABLE IF EXISTS record_fts
# ;

# -- CREATE VIRTUAL TABLE record USING fts5(title,pub_date,pub_name,pub_year,subjects)
# --;

# --INSERT INTO 
# --    record_fts (title, pub_date, pub_name, pub_year, subjects)
# --SELECT
# --    record.title,
# --    record.pub_date,
# --    record.pub_name,
# --    record.pub_year,
# --    record.subjects
# --FROM
# --    record
# --;

# --ANALYZE
# --;
# """

# with Session(engine) as session:
#     for statement in sql.split(';'):
#         session.execute(text(statement))
    
#     session.commit()

In [12]:
# TODO: convert these to the Python API 
!sqlite-utils --version

!sqlite-utils disable-fts record_data.db record

# enable full-text search 
!sqlite-utils enable-fts record_data.db record title pub_date pub_name pub_year subjects --create-triggers

!sqlite-utils query record_data.db "DROP INDEX IF EXISTS idx_record_pub_year"
!sqlite-utils create-index record_data.db record pub_year --analyze

!sqlite-utils query record_data.db "DROP INDEX IF EXISTS idx_record_pub_name"
!sqlite-utils create-index record_data.db record pub_name --analyze

!sqlite-utils query record_data.db "DROP INDEX IF EXISTS idx_record_title"
!sqlite-utils create-index record_data.db record title --analyze

!sqlite-utils query record_data.db "DROP INDEX IF EXISTS idx_record_pub_date"
!sqlite-utils create-index record_data.db record pub_date --analyze

!sqlite-utils index-foreign-keys record_data.db

sqlite-utils, version 3.30
[{"rows_affected": -1}]
[{"rows_affected": -1}]
[{"rows_affected": -1}]
[{"rows_affected": -1}]


In [3]:
db = sqlite_utils.Database('record_data.db')

# http://newsdex.plch.net/Newsdex+Record+Data/record?_sort=pub_year&subjects__arraycontains=World+War+II

db.create_view(
    name="subject_search_view",
    replace=True,
    sql="""\
with subject_values as (
  select
    value,
    count(*) as count_values
  from
    record,
    json_each([record].[subjects])
  group by
    1
)
select
  -- value,
  json_object(
    "href",
    "/Newsdex+Record+Data/record?subjects__arraycontains=" 
        || replace(replace(replace(replace(replace(value ,
            " ", "%20"),
            "/", "%2F"),
            "(", "%28"),
            ")", "%29"),
            "&", "%26")
        || "&_sort=pub_year",
    "label",
    value,
    "description",
    count_values || " Records containing subject '" || value || "'" 
  ) as "indexed subject",
  count_values as "count records"
from
  subject_values
"""
)


db.create_view(
    name="title_search_view",
    replace=True,
    sql="""
with title_values as (
  select
    coalesce(title, '') as title,
    count(*) as count_titles
  from
    record
  group by
    1
)
select
  -- value,
  json_object(
    "href",
    "/Newsdex+Record+Data/record?title=" 
        || replace(replace(replace(replace(replace(title ,
            " ", "%20"),
            "/", "%2F"),
            "(", "%28"),
            ")", "%29"),
            "&", "%26")
        || "&_sort=pub_year",
    "label",
    title,
    "description",
    count_titles || " Records containing title '" || title || "'" 
  ) as "indexed titles",
  count_titles as "count records"
from
  title_values;
"""
)



db.create_view(
    name="publication_search_view",
    replace=True,
    sql="""
with pub_values as (
  select
    coalesce(pub_name, '') as pub_name,
    count(*) as count_records
  from
    record
  group by
    1
)
select
  -- value,
  json_object(
    "href",
    "/Newsdex+Record+Data/record?pub_name=" 
        || replace(replace(replace(replace(replace(pub_name ,
            " ", "%20"),
            "/", "%2F"),
            "(", "%28"),
            ")", "%29"),
            "&", "%26")
        || "&_sort=pub_year",
    "label",
    pub_name,
    "description",
    count_records || " Records in publication '" || pub_name || "'" 
  ) as "indexed records",
  count_records as "count records"
from
  pub_values;
"""
)


db.create_view(
    name="published_year_search_view",
    replace=True,
    sql="""
with pub_values as (
  select
    coalesce(pub_year, '') as pub_year,
    count(*) as count_records
  from
    record
  group by
    1
)
select
  -- value,
  json_object(
    "href",
    "/Newsdex+Record+Data/record?pub_year=" 
        || replace(replace(replace(replace(replace(pub_year ,
            " ", "%20"),
            "/", "%2F"),
            "(", "%28"),
            ")", "%29"),
            "&", "%26")
        || "&_sort=pub_date",
    "label",
    pub_year,
    "description",
    count_records || " Records in published in year '" || pub_year || "'" 
  ) as "indexed records",
  count_records as "count records"
from
  pub_values;
"""
)

<Database <sqlite3.Connection object at 0x7f9ad366b740>>

In [13]:
datetime.strptime(record['773']['g'].format()[:10], '%m/%d/%Y')

datetime.datetime(1900, 10, 2, 0, 0)

In [14]:
record['773']['g'].format()

'10/02/1900 6:6'

In [15]:
record['773']['g'].format()[10:].strip()

'6:6'

In [16]:
for field in record.fields:
    print(field.tag, field.value(), end="\n")
    # print([(field.tag, field.value()) for field in record.fields], sep="\n")

008                                         
245 Public library staff: Instructive talks heard and a social hour enjoyed by employees
260 1900
650 Public Library
650 Public Library, Personnel
650 Hodges, Nathaniel Dana Carlisle
773 Commercial Tribune 10/02/1900 6:6
907 .b28911532 03-06-20 03-06-20
998 2ma 03-06-20 m - - eng 0


In [17]:
record['907'].subfields_as_dict()['c']

['03-06-20']

In [18]:
# [subfield for subfield in record['773'].get_subfields]

In [19]:
record.as_json()

'{"leader": "00467nam  2200133 a 4500", "fields": [{"008": "                                        "}, {"245": {"subfields": [{"a": "Public library staff: Instructive talks heard and a social hour enjoyed by employees"}], "ind1": " ", "ind2": "0"}}, {"260": {"subfields": [{"c": "1900"}], "ind1": " ", "ind2": " "}}, {"650": {"subfields": [{"a": "Public Library"}], "ind1": " ", "ind2": " "}}, {"650": {"subfields": [{"a": "Public Library, Personnel"}], "ind1": " ", "ind2": " "}}, {"650": {"subfields": [{"a": "Hodges, Nathaniel Dana Carlisle"}], "ind1": " ", "ind2": " "}}, {"773": {"subfields": [{"a": "Commercial Tribune"}, {"g": "10/02/1900 6:6"}], "ind1": " ", "ind2": " "}}, {"907": {"subfields": [{"a": ".b28911532"}, {"b": "03-06-20"}, {"c": "03-06-20"}], "ind1": " ", "ind2": " "}}, {"998": {"subfields": [{"a": "2ma"}, {"b": "03-06-20"}, {"c": "m"}, {"d": "-"}, {"e": "-"}, {"f": "eng"}, {"h": "0"}], "ind1": " ", "ind2": " "}}]}'

In [20]:
# with Session(engine) as session:
#     with zipfile.ZipFile('2023-01-24_newsdex-full-marc-export.zip') as marc_data_file:
#         # should only contain one file ...
#         print(marc_data_file.filelist[0].filename)
#         # get a file handle for the marc data file
#         with marc_data_file.open(marc_data_file.filelist[0]) as fh:
#             reader = MARCReader(fh)
#             for i, record in enumerate(reader):
#                 subjects = []
#                 for j, s in enumerate([subject.value() for subject in record.subjects()]):
#                     subjects.append(Subject(value=s, occ_num=j))
#                 r = Record(
#                     title=record.title(),
#                     record_json=record.as_json(),
#                     subjects=subjects
#                 )

#                 session.add(r)
#                 if (i % 100000 == 0 ):
#                     print(f'{i}.', end='')
#                     session.commit()

#                 # print(i, record['001'].value(), record.title(), [subject.value() for subject in record.subjects()])
#                 # if i>=100:
#                 #     break
#     session.commit()

In [32]:
# metadata.json

popular_subjects = """\
select
  -- record.id as record_id,
  json_object(
    "href",
    "/Newsdex+Record+Data/record?subjects__arraycontains=" 
        || replace(replace(replace(replace(replace(json_each.value ,
            " ", "%20"),
            "/", "%2F"),
            "(", "%28"),
            ")", "%29"),
            "&", "%26")
        || "&_sort=pub_year",
    "label",
    json_each.value,
    "description",
    "Records containing subject: '" || json_each.value || "'"
  ) as "indexed subject",
  --
  -- json_each.value as indexed_subject,
  count(record.id) as count_records
from
  record,
  json_each(subjects)
group by
  1
order by
  2 DESC;
"""

metadata = {
    "title": "CHPL Newsdex 📰",
    "extra_css_urls": [
      "/static/my.css"
    ],
    "databases": {
      "Newsdex Record Data": {
        "queries":{
          "popular subjects":{
            "sql": popular_subjects
          }
        }
      }
    }
}


with open("datasette/my-app/metadata.json", "w") as outfile:
    json.dump(metadata, outfile)

In [29]:
json.dumps(metadata)

'{"title": "CHPL Newsdex \\ud83d\\udcf0", "extra_css_urls": ["/static/my.css"], "databases": {"Newsdex Record Data": {"queries": {"popular subjects": {"sql": "CREATE VIEW subject_search_view AS select\\n  -- record.id as record_id,\\n  json_object(\\n    \\"href\\",\\n    \\"/Newsdex+Record+Data/record?subjects__arraycontains=\\" \\n        || replace(replace(replace(replace(json_each.value ,\\n            \\" \\", \\"%20\\"),\\n            \\"/\\", \\"%2F\\"),\\n            \\"(\\", \\"%28\\"),\\n            \\")\\", \\"%29\\")\\n        || \\"&_sort=pub_year\\",\\n    \\"label\\",\\n    json_each.value,\\n    \\"description\\",\\n    \\"Records containing subject: \'\\" || json_each.value || \\"\'\\"\\n  ) as \\"indexed subject\\",\\n  --\\n  -- json_each.value as indexed_subject,\\n  count(record.id) as count_records\\nfrom\\n  record,\\n  json_each(subjects)\\ngroup by\\n  1\\norder by\\n  2 DESC;\\n"}}}}}'